In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np; np.set_printoptions(precision=2); np.random.seed(0)
import torch; torch.set_printoptions(precision=2)
seed = 1

torch.manual_seed(seed)
import torch.nn as nn
import matplotlib.pyplot as plt
# import matplotlib 
from matplotlib.font_manager import FontProperties
from mpl_toolkits import mplot3d
import matplotlib.pylab as pl
from mpltern.ternary.datasets import get_scatter_points


import seaborn as sns
import time
import sys
import itertools 
import random; random.seed(0)
import datetime
import pickle
import copy
import pandas as pd
import scipy; from scipy import stats; from scipy.stats import wilcoxon
import os

from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

import sys
from functions import *


print(torch.__version__)
print(sys.version)
                
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1.13.1+cu116
3.10.8 (main, Nov 24 2022, 14:13:03) [GCC 11.2.0]


# Figure 6e-g and Supplementary Figure 10c-e

In [ ]:
with open('/.../conn_bias_sm_w_rec.pickle', 'rb') as handle:
    conn_bias_sr_all_models = pickle.load(handle)


data_fig6e = dict.fromkeys(['subtractive', 'divisive_2'])     # connectivity biases between rule-selective populations
for key in data_fig6e.keys():
    data_fig6e[key] = {'exc_exc': [], 'exc_pv': [], 'pv_exc': [], 'pv_pv': []}
    
data_fig6f = dict.fromkeys(['subtractive', 'divisive_2'])    # connectivity biases between response location-selective populations
for key in data_fig6e.keys():
    data_fig6f[key] = {'exc_exc': [], 'exc_pv': [], 'pv_exc': [], 'pv_pv': []}
    
data_fig6g = dict.fromkeys(['subtractive', 'divisive_2'])    # connectivity biases between shared feature-selective populations
for key in data_fig6e.keys():
    data_fig6g[key] = {'exc_exc': [], 'exc_pv': [], 'pv_exc': [], 'pv_pv': []}



for dend_nonlinear in ['subtractive', 'divisive_2']:
    fig, ax = plt.subplots(figsize=[10, 5])
    fig.suptitle('Connectivity biases across all models\n{}'.format(dend_nonlinear), fontsize=20)
    fig.patch.set_facecolor('white')
    for x in conn_bias_sr_all_models:
        if x['hp']['dend_nonlinearity'] != dend_nonlinear:
            continue
        data = list(x.values())[2:6]
        if np.isnan(data).any():
            continue
    
        data_fig6e[dend_nonlinear]['exc_exc'].append(data[0])
        data_fig6e[dend_nonlinear]['exc_pv'].append(data[1])
        data_fig6e[dend_nonlinear]['pv_exc'].append(data[2])
        data_fig6e[dend_nonlinear]['pv_pv'].append(data[3])
    
        
        ax.plot(data, marker='o', color='k', linewidth=2, markersize=10, alpha=0.5, clip_on=False)
    ax.set_xticks(np.arange(len(data)))
    ax.set_xticklabels([r'rule E $\rightarrow$ rule E', r'rule E $\rightarrow$ rule PV', r'rule PV $\rightarrow$ rule E', r'rule PV $\rightarrow$ rule PV'], rotation=10)
    ax.axhline(y=0, ls='--', color='k')
    ax.set_xlim(-0.5, len(data)-0.5)
    # ax.set_ylim(-1,2)
    ax.set_ylabel('Connectivity bias', fontsize=20)
    make_pretty_axes(ax)
    fig.tight_layout()
    plt.show()
    
    
    fig, ax = plt.subplots(figsize=[10, 5])
    fig.suptitle('Connectivity biases across all models\n{}'.format(dend_nonlinear), fontsize=20)
    fig.patch.set_facecolor('white')
    for x in conn_bias_sr_all_models:
        if x['hp']['dend_nonlinearity'] != dend_nonlinear:
            continue
        data = list(x.values())[6:10]
        if np.isnan(data).any():
            continue
    
    
        data_fig6f[dend_nonlinear]['exc_exc'].append(data[0])
        data_fig6f[dend_nonlinear]['exc_pv'].append(data[1])
        data_fig6f[dend_nonlinear]['pv_exc'].append(data[2])
        data_fig6f[dend_nonlinear]['pv_pv'].append(data[3])
    
        
        ax.plot(data, marker='o', color='k', linewidth=2, markersize=10, alpha=0.5, clip_on=False)
    ax.set_xticks(np.arange(len(data)))
    ax.set_xticklabels([r'resp E $\rightarrow$ resp E', r'resp E $\rightarrow$ resp PV', r'resp PV $\rightarrow$ resp E', r'resp PV $\rightarrow$ resp PV'], rotation=10)
    ax.axhline(y=0, ls='--', color='k')
    ax.set_xlim(-0.5, len(data)-0.5)
    # ax.set_ylim(-1,2)
    ax.set_ylabel('Connectivity bias', fontsize=20)
    make_pretty_axes(ax)
    fig.tight_layout()
    plt.show()
    
    
    fig, ax = plt.subplots(figsize=[10, 5])
    fig.suptitle('Connectivity biases across all models\n{}'.format(dend_nonlinear), fontsize=20)
    fig.patch.set_facecolor('white')
    for x in conn_bias_sr_all_models:
        if x['hp']['dend_nonlinearity'] != dend_nonlinear:
            continue
        data = list(x.values())[14:18]
        if np.isnan(data).any():
            continue
    
        data_fig6g[dend_nonlinear]['exc_exc'].append(data[0])
        data_fig6g[dend_nonlinear]['exc_pv'].append(data[1])
        data_fig6g[dend_nonlinear]['pv_exc'].append(data[2])
        data_fig6g[dend_nonlinear]['pv_pv'].append(data[3])
        
        ax.plot(data, marker='o', color='k', linewidth=2, markersize=10, alpha=0.5, clip_on=False)
    ax.set_xticks(np.arange(len(data)))
    ax.set_xticklabels([r'shared feature E $\rightarrow$ E', r'shared feature E $\rightarrow$ PV', r'shared feature PV $\rightarrow$ E', r'shared feature PV $\rightarrow$ PV'], rotation=10)
    ax.axhline(y=0, ls='--', color='k')
    ax.set_xlim(-0.5, len(data)-0.5)
    # ax.set_ylim(-1,2)
    ax.set_ylabel('Connectivity bias', fontsize=20)
    make_pretty_axes(ax)
    fig.tight_layout()
    plt.show()

    
    